# chiTraモデルのファインチューニング

前のノートブックで見たように、配布されているchiTraモデルは「マスクされた語を予測する」ように学習された状態で、そのままでは他のタスクには利用できません。
そこで、応用したいタスクのデータに対して望む出力ができるよう調整を行います。これがファインチューニングと呼ばれる作業です。

このノートブックでは、
- chiTraモデルを実際のタスクに合わせてファインチューニングし、その効果を確認すること
- 訓練したモデルをデプロイし、実際に使ってみること

がゴールです。

## 準備

### 出力の消去
配布のノートブックでは、各セルの実行結果を参照用に残しています。

作業においては実行場所がわかりにくくなるので、右クリックのメニューから`Clear All Outputs`を実行して消去します。

本来どのようになるのか確認したい、初期状態に戻したいなどの場合は、
ターミナルからコマンド `tar -xvf notebooks.tar.gz` で再度展開を行ってください（ファイルの上書きにご注意ください）。

### 定数
chiTraモデルデータへのパスのほか、AWS内のリソースへのパスなど、
ノートブック内で使用する定数をいくつか定義します

In [1]:
chitra_path = "./chiTra-1.0"

s3_handson_bucket = "chitra-handson-20221203"
s3_source_path = f"s3://{s3_handson_bucket}/source/sourcedir.tar.gz"
s3_data_path = f"s3://{s3_handson_bucket}/datasets/livedoor"
s3_output_path = f"s3://{s3_handson_bucket}/trained"

rawdata_path = f"{s3_data_path}/raw"
train_input_path = f"{s3_data_path}/train"
test_input_path = f"{s3_data_path}/test"

## タスクデータの準備

今回は [livedoor ニュースコーパス](https://www.rondhuit.com/download.html#ldcc)を利用します。
こちらは「livedoor ニュース」の９つのニュースカテゴリから記事を収集したものです。

記事本文の内容を入力として、どのカテゴリの記事かを判定するタスクを対象としてモデルをチューニングしていきます。

### データの確認

まずはデータセットの内容を少し見てみましょう。

In [2]:
# s3 からデータをダウンロード
import datasets
import botocore
from datasets.filesystems import S3FileSystem

s3 = S3FileSystem()
livedoor_data = datasets.load_from_disk(rawdata_path, fs=s3)

In [3]:
# データの量を確認
print(livedoor_data)

Dataset({
    features: ['sentence1', 'label'],
    num_rows: 7367
})


In [4]:
# カテゴリラベルの一覧
labels = list(sorted(set(livedoor_data["label"])))
print("ラベル一覧", labels)

ラベル一覧 ['dokujo-tsushin', 'it-life-hack', 'kaden-channel', 'livedoor-homme', 'movie-enter', 'peachy', 'smax', 'sports-watch', 'topic-news']


In [5]:
# データをランダムに表示
data = livedoor_data.shuffle()[0]

print("ラベル:", data['label'])
print("本文:", f"{data['sentence1'][:400]}...")

ラベル: it-life-hack
本文: 戻ってまいりました！先日あえなく砕け散ったインテルの最新SSD「520シリーズ」を旧型Macに取りつけるという連載。今回から装いも新たに、かつ名誉挽回を目指し前回とは異なるアプローチで装着させることを試みた。旧MacがSSDによる「快適動作」に成功するかどうか、乞うご期待。 ■SSD外付けでの起動にアプローチ 以前の「インテル SSD 520を旧Macに装着」シリーズは、結局、MacProでの動作に成功しないまま終了してしまった。筆者はSATAコントローラに問題があると推測したが、発売後５年以上を経た今、コントローラの改善は期待できない。そこで、SSDを既存のHDDスペースに装着する以外の方法を試してみることにした。 Macは、Windowsと異なり、外付け機器からの起動を広くサポートしている。標準で、USB接続の外付けHDDやフラッシュメモリ、FireWire（IEEE1394）接続の外...


### データの加工

前のノートブックで見たように、chiTraモデルはテキストをそのまま扱うことはできないので、chiTraトークナイザを使って加工を行います。

また併せてカテゴリラベルについても数値へ変換します。

In [6]:
import sudachitra as chitra

# トークナイザを読み込む
tok = chitra.BertSudachipyTokenizer.from_pretrained(chitra_path)

# ラベルを数値に変換・逆変換する
label2id = {l:i for i,l in enumerate(labels)}
id2label = {i:l for i,l in enumerate(labels)}

# データをモデル入力用に整形する関数
def preprocess(data):
    # chiTraトークナイザを適用
    ret = tok(data["sentence1"], padding=True, truncation=True, max_length=512)
    # ラベルを変換
    ret["label"] = [label2id[l] for l in data["label"]]
    return ret

# 加工の実行（ここでは一部のみ）
processed_data = livedoor_data.shuffle().select(range(100)).map(
    preprocess,
    batched=True,
    remove_columns=["sentence1"]
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
# 加工済みデータをランダムに表示
print(processed_data.shuffle()[0])

{'label': 1, 'input_ids': [2, 19559, 18763, 484, 10309, 1432, 16694, 2476, 485, 418, 16295, 519, 11254, 476, 14922, 480, 12116, 519, 461, 476, 10146, 419, 12861, 1083, 6514, 484, 11316, 6139, 6740, 481, 10653, 11632, 450, 11098, 477, 12903, 478, 10152, 476, 10146, 419, 10209, 485, 418, 13962, 484, 1106, 5388, 10148, 11499, 469, 419, 1432, 11476, 485, 418, 427, 23905, 11477, 23488, 22031, 484, 10992, 11791, 477, 418, 19559, 484, 11316, 6139, 6740, 484, 12370, 12244, 459, 514, 469, 12610, 450, 11187, 459, 514, 476, 442, 11439, 419, 27044, 12135, 11370, 450, 15154, 481, 12244, 461, 11439, 419, 63, 30620, 5305, 6507, 7312, 7312, 69, 5346, 5298, 5305, 16, 24995, 17, 63, 5300, 5537, 20, 29118, 428, 478, 418, 28064, 5810, 5497, 469, 419, 10154, 1106, 5388, 481, 10272, 476, 418, 20734, 484, 13307, 450, 14098, 10158, 476, 10146, 419, 427, 11847, 461, 476, 442, 11439, 428, 427, 693, 481, 616, 5649, 15574, 10811, 10152, 476, 504, 15621, 485, 1214, 481, 13242, 476, 781, 10400, 2052, 6852, 461, 476

### データのアップロード

今回は準備済みなので必要ありませんが、一から準備を行う場合はさらに、
訓練用と確認用のデータを分割し、s3へとアップロードしておくことになります。

In [8]:
# import botocore
# from datasets.filesystems import S3FileSystem

# split = processed_data.train_test_split(test_size=0.2)

# s3 = S3FileSystem()
# split["train"].save_to_disk(f"{train_input_path}_tmp", fs=s3)
# split["test"].save_to_disk(f"{test_input_path}_tmp", fs=s3)

## ファインチューニング

次に加工したデータを使ってモデルのファインチューニングを行うジョブを実行していきます。

### SageMakerで訓練を行う

SageMaker SDKライブラリを用いることで、ノートブックから訓練ジョブを作成することができます。

APIを叩く形になるためここでのコードはシンプルで、事前に用意したリソースや訓練の細かい設定を引数として渡していくことになります。

学習には少々時間がかかるので、まずはジョブを実行してしまいましょう。

In [9]:
import sagemaker
from sagemaker.huggingface import HuggingFace

# SageMakerへのセッションを確立
sess = sagemaker.Session(default_bucket=s3_handson_bucket)

# 訓練ジョブの定義
huggingface_estimator = HuggingFace(
    sagemaker_session=sess,          # セッション
    output_path=s3_output_path,      # 訓練結果の出力先
    source_dir=s3_source_path,       # 訓練用リソースの置き場所（後で説明）
    entry_point='train.py',          # 訓練スクリプトの名称
    instance_type='ml.g4dn.xlarge',  # 学習に使用するインスタンス
    instance_count=1,
    role="arn:aws:iam::012345678901:role/ChitraHandsonSageMakerExecution",
    transformers_version='4.12',
    pytorch_version='1.9',
    py_version='py38',
    hyperparameters = {
        'num_labels': len(labels), # 今回のタスクのラベル数
        'epochs': 1,               # チューニングの量
        'train-batch-size': 12,
    },
)

In [10]:
# 使用するデータを指定して訓練を始める
huggingface_estimator.fit({
    'train': train_input_path,  # 訓練用のデータ
    'test': test_input_path,    # 精度確認用のデータ
})

# 後から参照するため訓練ジョブの名称を控えておく
job_name = huggingface_estimator.latest_training_job.job_name
print("ジョブの名称:", job_name)

2022-11-30 10:05:46 Starting - Starting the training job...
2022-11-30 10:06:09 Starting - Preparing the instances for trainingProfilerReport-1669802745: InProgress
............
2022-11-30 10:08:18 Downloading - Downloading input data
2022-11-30 10:08:18 Training - Downloading the training image............
2022-11-30 10:10:30 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-11-30 10:10:36,000 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-11-30 10:10:36,026 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-11-30 10:10:36,029 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-11-30 10:10:40,836 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python -m pip install -r requirements.tx

### 訓練用リソースについて

上で訓練用ジョブを定義する際、`source_dir = "s3://..."` という引数で訓練用リソースを渡していました。

この s3 arn が指しているのは、ハンズオンの最初にダウンロードした `sourcedir.tar.gz` と同じもの（つまりノートブック以外のファイル全て）です。

訓練の間にこれらについて内容を説明していきます。

### チューニング結果の確認

訓練の結果、最終的にどの程度の精度になったのかを確認します。

ログの最後の方、"***** Eval results *****" 以降にファインチューニング後のモデル評価結果が出力されています。
`eval_accuracy = ...` の部分がテストデータでの精度です。
学習には乱数が関わるため一定ではありませんが、およそ 90%の精度になっているかと思います。

今回のタスクには９つのラベルがありました。
ファインチューニングなしのモデルでは出力ラベルがランダムなため、精度は 1/9、すなわち11%程度となります。
これを踏まえると、ファインチューニングによってモデルがタスクに適応し、高い精度が得られるようになっていることがわかります。

なお今回は訓練データ一周分 (1 epoch) の訓練を行いましたが、この量やそのほかの学習パラメータを調整することで更に高い精度が得られる可能性もあります。
この、タスクに最適なパラメータを探す作業をハイパーパラメータ探索と呼びますが、本ハンズオンでは詳細は割愛します。

## 訓練したモデルのデプロイ

最後に、先ほど訓練したモデルをデプロイして、実際にテキストのラベルを予測させてみましょう。

こちらも訓練の時と同様、SageMakerのAPIを呼ぶ形になります。

### SageMakerでモデルをデプロイする

In [11]:
# 訓練ジョブの名称でデプロイする訓練済みモデルを指定
# もしくは上の `huggingface_estimator` をそのまま使うことも可能
estimator = HuggingFace.attach(job_name, sagemaker_session=sess)

# モデルをデプロイする
predictor = estimator.deploy(1, "ml.g4dn.xlarge")


2022-11-30 10:23:34 Starting - Preparing the instances for training
2022-11-30 10:23:34 Downloading - Downloading input data
2022-11-30 10:23:34 Training - Training image download completed. Training in progress.
2022-11-30 10:23:34 Uploading - Uploading generated training model
2022-11-30 10:23:34 Completed - Training job completed
-----------!

### ラベルを予測させる
タスクデータからいくつかサンプルをとって、ラベルを予測させてみましょう。

In [12]:
# ラベルごとの予測スコアを表示する関数
def print_label_score(pred):
    print("各ラベルのスコア:")
    for l, score in sorted(enumerate(pred["logits"][0]), key=lambda x: -x[1]):
        print(f"{id2label[l]}\t", f"{score:.2f}")

In [13]:
# ランダムなデータを選択
d = livedoor_data.shuffle()[0]
text = d["sentence1"]
label = d["label"]

print("真のラベル:", label)
print(f"本文: {text[:200]}...\n")

# モデルにラベルを予測させる
pred = predictor.predict({"inputs": text})

print("予測ラベル:", id2label[pred["label"]])
print_label_score(pred)

真のラベル: sports-watch
本文: 野球解説者、野村克也氏が今年の野球界を漢字一文字で表すと？ TBSのスポーツ番組「S1」（25日深夜放送）も年内最後の放送となり、人気コーナー「ノムさんぼやき部屋」もこの日の放送でボヤキおさめとなった。そんな今年最後のテーマは前述した通り——。 登場するや、TBS枡田絵理奈アナウンサーに「誕生日おめでとう」と花束を渡したノムさん。12月25日で26歳になった枡田アナに「36歳か？」というボケをかま...

予測ラベル: sports-watch
各ラベルのスコア:
sports-watch	 4.21
topic-news	 4.06
it-life-hack	 0.21
livedoor-homme	 -0.58
movie-enter	 -0.64
kaden-channel	 -0.93
smax	 -1.23
peachy	 -1.63
dokujo-tsushin	 -2.08


In [14]:
# 新規の文章を入力とする
text = "吾輩は猫である。"

# モデルにラベルを予測させる
pred = predictor.predict({"inputs": text})

print("予測ラベル:", id2label[pred["label"]])
print_label_score(pred)

予測ラベル: dokujo-tsushin
各ラベルのスコア:
dokujo-tsushin	 1.83
kaden-channel	 0.76
livedoor-homme	 0.59
it-life-hack	 0.20
sports-watch	 -0.24
movie-enter	 -0.29
smax	 -0.48
topic-news	 -1.04
peachy	 -1.12


### 後処理

最後にデプロイしたインスタンスを削除します。

In [15]:
# 間違えて削除してしまわないようコメントアウトしている
# 行頭の "#" を削除してから実行する

# predictor.delete_endpoint()